# Import packages and own modules

In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor

import matplotlib.pyplot as plt
import seaborn as sns

import holidays

import sys
sys.path += ['../src']
from feature_engineering import *
from feature_scaling import *
from outliers import *
from utils import *
from helperFunctions import *

%matplotlib inline

# 1. Load input data
## 1.1 Load and merge data

In [3]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/holdout.csv")

store = pd.read_csv("../data/store.csv")
data_train = pd.merge(store, train, on='Store', how='left')
data_test = pd.merge(store, train, on='Store', how='left')

/Applications/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 1.2 Convert 'Date' column and generate generic date features

In [4]:
data_train.loc[:, 'Date'] = pd.to_datetime(data_train.loc[:, 'Date'])
data_train.loc[:, 'Month'] = data_train['Date'].dt.month
data_train.loc[:, 'DayOfMonth'] = data_train['Date'].dt.day
data_train.loc[:, 'Year'] = data_train['Date'].dt.year
data_train.loc[:, 'DayOfWeek'] = data_train['Date'].dt.dayofweek
data_train.loc[:, 'WeekOfYear'] = data_train['Date'].dt.weekofyear

data_test.loc[:, 'Date'] = pd.to_datetime(data_test.loc[:, 'Date'])
data_test.loc[:, 'Month'] = data_test['Date'].dt.month
data_test.loc[:,'DayOfMonth'] = data_test['Date'].dt.day
data_test.loc[:,'Year'] = data_test['Date'].dt.year
data_test.loc[:,'DayOfWeek'] = data_test['Date'].dt.dayofweek
data_test.loc[:,'WeekOfYear'] = data_test['Date'].dt.weekofyear

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  del sys.path[0]


## 1.3 Drop rows based on 'Sales' column 
#### Drop rows with value zero

In [5]:
zero_sales = data_train.Sales == 0
data_train = data_train.loc[~zero_sales]
print(f'Drop {sum(zero_sales)} rows, keep {data_train.shape[0]}')

zero_sales = data_test.Sales == 0
missing_sales = data_test.Sales.isnull()
data_test = data_test.loc[~(zero_sales|missing_sales)]
print(f'Drop {sum(zero_sales|missing_sales)} rows, keep {data_test.shape[0]}')

Drop 102652 rows, keep 515821
Drop 121097 rows, keep 497376


## 1.4 Remove erroneous data
#### Drop row with nonzero sales despite being closed

In [6]:
data_train = closed_sales(data_train)
data_test = closed_sales(data_train)

## 1.5 Compute 'Sales_per_customer' column
#### Compute sales per customer for each store from training set data

In [7]:
group = data_train.groupby(by='Store').agg({'Sales': 'mean', 'Customers': 'mean'})
group['Sales_per_customer'] = group['Sales'] / group['Customers']

#### Map sales per customer to training and test set

In [8]:
data_train['Sales_per_customer'] = data_train['Store'].map(group['Sales_per_customer'])
data_test['Sales_per_customer'] = data_test['Store'].map(group['Sales_per_customer'])

## 1.6 Compute 'Customers_per_store' column
#### Compute customers per store for each store from training set data

In [9]:
group = data_train.groupby('Store').agg({'Customers': 'mean'})

#### Map customers per store to training and test set

In [10]:
data_train['Customer_per_store'] = data_train['Store'].map(group['Customers'])
data_test['Customer_per_store'] = data_test['Store'].map(group['Customers'])

# 2. Missing value imputation
#### Overview before imputing missing values

In [11]:
missing_report(data_train, pd)

,Null (total),Null (percent),Type
Store,0,0.00,int64
StoreType,0,0.00,object
Assortment,0,0.00,object
CompetitionDistance,1351,0.26,float64
CompetitionOpenSinceMonth,162948,31.77,float64
CompetitionOpenSinceYear,162948,31.77,float64
Promo2,0,0.00,int64
Promo2SinceWeek,252634,49.26,float64
Promo2SinceYear,252634,49.26,float64
PromoInterval,252634,49.26,object


## 2.1 'Promo', 'Promo2'
#### Drop all null values

In [12]:
missing_promo = data_train.Promo.isnull() | data_train.Promo2.isnull()
data_train = data_train.loc[~missing_promo, :]
print(f'Drop {sum(missing_promo)} rows, keep {data_train.shape[0]}')

missing_promo = data_test.Promo.isnull() | data_test.Promo2.isnull()
data_test = data_test.loc[~missing_promo, :]
print(f'Drop {sum(missing_promo)} rows, keep {data_test.shape[0]}')

Drop 15432 rows, keep 497396
Drop 15432 rows, keep 497396


## 2.2 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear'
#### Apply mean imputation for missing values

In [13]:
data_train = mean_imputation(data_train,['CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear'])
data_test = mean_imputation(data_test,['CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear'])

## 2.3 'Promo2SinceWeek', 'Promo2SinceYear',  'PromoInterval'
#### Apply mean imputation for stores participating and constant imputation with zero for stores not participating

In [14]:
data_train = mean_imputation(data_train, ['Promo2SinceWeek', 'Promo2SinceYear'], enforce_int=True)
Promo2 = data_train.Promo2 == 1
data_train.loc[~Promo2, ['Promo2SinceWeek', 'Promo2SinceYear']] = 0
data_train = const_imputation(data_train,['PromoInterval'],values='unavailable')

data_test = mean_imputation(data_test, ['Promo2SinceWeek', 'Promo2SinceYear'], enforce_int=True)
Promo2 = data_test.Promo2 == 1
data_test.loc[~Promo2, ['Promo2SinceWeek', 'Promo2SinceYear']] = 0
data_test = const_imputation(data_test,['PromoInterval'],values='unavailable')

## 2.4 'Sales', 'Customers', 'Open'
#### Drop rows with missing 'Sales' in case 'Open' or 'Customers' is zero (training set only)

In [15]:
not_open = data_train.Open==0
no_customer = data_train.Customers==0
missing_sales = data_train.Sales.isnull()
data_train = data_train.loc[~((not_open | no_customer) & missing_sales), :]

print(f'Drop {sum(((not_open | no_customer) & missing_sales))} rows, keep {data_train.shape[0]}')

Drop 85 rows, keep 497311


#### Impute 'Sales' from 'Customers' and 'Sales_per_customer' (training set only)

In [16]:
missing_sales = data_train.Sales.isnull()
data_train.loc[missing_sales, 'Sales'] = data_train.loc[missing_sales, 'Customers'] * data_train.loc[missing_sales, 'Sales_per_customer']
missing_sales = data_train.Sales.isnull()
data_train = data_train.loc[~missing_sales, :]

print(f'Drop {sum(missing_sales)} rows, keep {data_train.shape[0]}')
data_train = const_imputation(data_train,['Open'],values=1)

Drop 451 rows, keep 496860


#### Drop rows with missing or zero 'Open' (test set only)

In [17]:
not_open = data_test.Open==0
missing_open = data_test.Open.isna()

data_test = data_test.loc[~(not_open | missing_open), :]

print(f'Drop {sum(not_open | missing_open)} rows, keep {data_test.shape[0]}')

Drop 15070 rows, keep 482326


## 2.5 'StateHoliday'
#### Look up if 'Date' is a national holiday and impute missing values with 'a' and '0' accordingly

In [18]:
data_train = holiday_imputation(data_train, holidays)
data_test = holiday_imputation(data_test, holidays)

## 2.6 'CompetitionDistance'
#### Drop rows with missing values

In [19]:
missing_distance = data_train.CompetitionDistance.isnull()
data_train = data_train.loc[~missing_distance, :]
print(f'Drop {sum(missing_distance)} rows, keep {data_train.shape[0]}')

missing_distance = data_test.CompetitionDistance.isnull()
data_test = data_test.loc[~missing_distance, :]
print(f'Drop {sum(missing_distance)} rows, keep {data_test.shape[0]}')

Drop 1299 rows, keep 495561
Drop 1246 rows, keep 481080


## 2.7 Drop columns 'Customers', 'Open' and 'SchoolHoliday'

In [20]:
data_train = data_train.drop(columns = ['Customers','Open', 'SchoolHoliday'])
data_test = data_test.drop(columns = ['Customers','Open', 'SchoolHoliday'])

In [21]:
missing_report(data_train, pd)

,Null (total),Null (percent),Type
Store,0,0.0,int64
StoreType,0,0.0,object
Assortment,0,0.0,object
CompetitionDistance,0,0.0,float64
CompetitionOpenSinceMonth,0,0.0,float64
CompetitionOpenSinceYear,0,0.0,float64
Promo2,0,0.0,int64
Promo2SinceWeek,0,0.0,float64
Promo2SinceYear,0,0.0,float64
PromoInterval,0,0.0,object


In [22]:
missing_report(data_test, pd)

,Null (total),Null (percent),Type
Store,0,0.0,int64
StoreType,0,0.0,object
Assortment,0,0.0,object
CompetitionDistance,0,0.0,float64
CompetitionOpenSinceMonth,0,0.0,float64
CompetitionOpenSinceYear,0,0.0,float64
Promo2,0,0.0,int64
Promo2SinceWeek,0,0.0,float64
Promo2SinceYear,0,0.0,float64
PromoInterval,0,0.0,object


# 3. Encoding of categorical variables
## 3.1 'Store'
#### Mean and frequency encoding

In [23]:
data_train, mean_values = mean_encoding(data_train, 'Store')
data_train, fenc_values = freq_encoding(data_train, 'Store')

data_test.loc[:, 'Store_menc'] = data_test['Store'].map(mean_values)
data_test.loc[:, 'Store_fenc'] = data_test['Store'].map(fenc_values)

## 3.2 'StoreType'
#### One hot encoding

In [24]:
data_train = pd.get_dummies(data_train, columns=['StoreType'], drop_first=True)
data_test = pd.get_dummies(data_test, columns=['StoreType'], drop_first=True)

## 3.3 'Assortment'
#### Ordinal encoding according to assortment levels: a = basic, b = extra, c = extended

In [25]:
data_train = ordinal_encoding(data_train, 'Assortment', {'a':1, 'b':2, 'c':3})
data_test = ordinal_encoding(data_test, 'Assortment', {'a':1, 'b':2, 'c':3})

## 3.4 'StateHoliday'
#### Binary encoding

In [26]:
data_train = binary_encoding(data_train, 'StateHoliday', positive_list=['a', 'b', 'c'])
data_test = binary_encoding(data_test, 'StateHoliday', positive_list=['a', 'b', 'c'])

## 3.5 Drop encoded columns and null values from test set

In [27]:
data_train = data_train.drop(columns=['Store', 'Assortment', 'StateHoliday'])
data_test = data_test.drop(columns=['Store', 'Assortment', 'StateHoliday'])
data_train = data_train.rename(columns={'Assortment_orenc': 'Assortment'})
data_test = data_test.rename(columns={'Assortment_orenc': 'Assortment'})
data_test = data_test.dropna()

# 4. Feature engineering
## 4.1 'CompetionOpenSincePeriod'

In [28]:
data_train = compet_openmonths(data_train, yr='CompetitionOpenSinceYear', mth='CompetitionOpenSinceMonth')
data_test = compet_openmonths(data_test, yr='CompetitionOpenSinceYear', mth='CompetitionOpenSinceMonth')

## 4.2 'Promo2Since'

In [29]:
data_train = promo_openweeks(data_train, yr='Promo2SinceYear', week='Promo2SinceWeek')
data_test = promo_openweeks(data_test, yr='Promo2SinceYear', week='Promo2SinceWeek')

## 4.3 Select features for prediction

In [30]:
my_features = ['Sales_per_customer', 'CompetitionDistance', 'Customer_per_store', 
               'Store_menc', 'CompetitionOpenSincePeriod', 
               'Month', 'Year', 'DayOfWeek', 'WeekOfYear', 'DayOfMonth', 
               'StoreType_b', 'StoreType_c', 'StoreType_d', 
               'Promo2SincePeriod', 'Promo2', 'Promo', 
               'HasCompetitor', 'CompetitionOpenSincePeriod', 
               'Assortment', 'CompetitionDistance', 'Sales']

In [31]:
data_train = data_train[my_features]
data_test = data_test[my_features]

In [32]:
X_train = data_train.drop(columns='Sales')
y_train = data_train['Sales']
X_test = data_test.drop(columns='Sales')
y_test = data_test['Sales']

In [33]:
X_train

,Sales_per_customer,CompetitionDistance,Customer_per_store,Store_menc,CompetitionOpenSincePeriod,Month,Year,DayOfWeek,WeekOfYear,DayOfMonth,StoreType_b,StoreType_c,StoreType_d,Promo2SincePeriod,Promo2,Promo,HasCompetitor,CompetitionOpenSincePeriod,Assortment,CompetitionDistance
1,8.336823,1270.0,574.903297,4792.602599,52.000000,1,2013,2,1,2,0,1,0,104677.0,0,0.0,1,52.000000,1,1270.0
2,8.336823,1270.0,574.903297,4792.602599,52.000000,1,2013,3,1,3,0,1,0,104677.0,0,0.0,1,52.000000,1,1270.0
3,8.336823,1270.0,574.903297,4792.602599,52.000000,1,2013,4,1,4,0,1,0,104677.0,0,0.0,1,52.000000,1,1270.0
4,8.336823,1270.0,574.903297,4792.602599,52.000000,1,2013,5,1,5,0,1,0,104677.0,0,0.0,1,52.000000,1,1270.0
6,8.336823,1270.0,574.903297,4792.602599,52.000000,1,2013,0,2,7,0,1,0,104678.0,0,1.0,1,52.000000,1,1270.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618467,14.364439,5350.0,406.749441,5830.536793,63.600505,7,2014,5,30,26,0,0,1,112.0,1,0.0,1,63.600505,3,5350.0
618469,14.364439,5350.0,406.749441,5830.536793,63.600505,7,2014,0,31,28,0,0,1,113.0,1,1.0,1,63.600505,3,5350.0
618470,14.364439,5350.0,406.749441,5830.536793,63.600505,7,2014,1,31,29,0,0,1,113.0,1,1.0,1,63.600505,3,5350.0
618471,14.364439,5350.0,406.749441,5830.536793,63.600505,7,2014,2,31,30,0,0,1,113.0,1,1.0,1,63.600505,3,5350.0
